In [22]:
from unittest import case

from qiskit import QuantumCircuit
from sympy.strategies.core import switch


class ClassicalCircuit:
    def __init__(self, filename):
        self.n_inputs = 0
        self.n_outputs = 0
        self.n_internal = 0
        self.input_gates = []  # list with n_inputs numbers indicating which are the input gates
        self.output_gates = []  # list with n_outputs numbers indicating which are the output gates
        self.internal_gates = []  # list with n_internal numbers indicating which are the internal gates
        self.gates = []
        self.read(filename)

    def read(self, filename):
        with open(filename, 'r') as file:
            lines = file.readlines()
        self.n_inputs = int(lines[0].strip())
        self.n_outputs = int(lines[1].strip())
        self.n_internal = int(lines[2].strip())
        self.input_gates = list(map(int, lines[3].strip().split()))
        self.output_gates = list(map(int, lines[4].strip().split()))
        self.internal_gates = list(map(int, lines[5].strip().split()))
        for line in lines[6:]:
            gate = line.strip().split()
            gate[0] = int(gate[0])
            if gate[1] == "and" or gate[1] == "or" or gate[1] == "xor" or gate[1] == "nand":
                gate[2] = int(gate[2])
                gate[3] = int(gate[3])
            elif gate[1] == "not":
                gate[2] = int(gate[2])
            self.gates.append(gate)

    def print(self):
        print(f"n_inputs: {self.n_inputs}")
        print(f"n_outputs: {self.n_outputs}")
        print(f"n_internal: {self.n_internal}")
        print(f"Input Gates: {self.input_gates}")
        print(f"Output Gates: {self.output_gates}")
        print(f"Internal Gates: {self.internal_gates}")
        print("Gates: ")  # reads n_outputs + n_internal
        for gate in self.gates:
            print(gate)
        print()

    def convert_step_1(self, quantumCircuit):
        for gate in self.gates:
            match gate[1]:
                case "and":
                    quantumCircuit.ccx(gate[2], gate[3], gate[0])
                case "not":
                    quantumCircuit.x(gate[0])
                    quantumCircuit.cx(gate[2], gate[0])
            quantumCircuit.barrier()

    def convert_step_2(self, quantumCircuit):
        for gate in self.gates[::-1]:
            match gate[1]:
                case "and":
                    quantumCircuit.ccx(gate[2], gate[3], gate[0])
                case "not":
                    quantumCircuit.cx(gate[2], gate[0])
                    quantumCircuit.x(gate[0])
            quantumCircuit.barrier()

    def convert(self, quantumCircuit):
        self.convert_step_1(quantumCircuit)
        output_bit_index = 0
        for gate in self.gates:
            if gate[0] in self.output_gates:
                quantumCircuit.cx(gate[0], self.n_inputs + self.n_internal + +self.n_outputs + output_bit_index)
                output_bit_index = output_bit_index + 1

        quantumCircuit.barrier()

        self.convert_step_2(quantumCircuit)

    def convert_contr_step_1(self,quantumCircuit,i):
        for gate in self.gates:
            match gate[1]:
                case "and":
                    quantumCircuit.ccx(gate[2], gate[3], gate[0])
                case "not":
                    quantumCircuit.x(gate[0])
                    quantumCircuit.cx(gate[2], gate[0])
                    pass
            quantumCircuit.barrier()


cc = ClassicalCircuit("circuit.txt")
cc.print()

n_wires = cc.n_inputs + cc.n_outputs + cc.n_internal
qc = QuantumCircuit(n_wires, 0)
cc.convert_step_1(qc)
print(qc)
print()

n_wires = cc.n_inputs + cc.n_outputs + cc.n_internal
qc = QuantumCircuit(n_wires, 0)
cc.convert_step_2(qc)
print(qc)
print()

n_wires = cc.n_inputs + 2 * cc.n_outputs + cc.n_internal
qc = QuantumCircuit(n_wires, 0)
cc.convert(qc)
print(qc)
print()

n_inputs: 3
n_outputs: 2
n_internal: 2
Input Gates: [0, 1, 2]
Output Gates: [3, 4]
Internal Gates: [5, 6]
Gates: 
[5, 'and', 0, 1]
[3, 'not', 5]
[6, 'not', 2]
[4, 'and', 5, 6]

           ░            ░            ░       ░ 
q_0: ──■───░────────────░────────────░───────░─
       │   ░            ░            ░       ░ 
q_1: ──■───░────────────░────────────░───────░─
       │   ░            ░            ░       ░ 
q_2: ──┼───░────────────░────────■───░───────░─
       │   ░ ┌───┐┌───┐ ░        │   ░       ░ 
q_3: ──┼───░─┤ X ├┤ X ├─░────────┼───░───────░─
       │   ░ └───┘└─┬─┘ ░        │   ░ ┌───┐ ░ 
q_4: ──┼───░────────┼───░────────┼───░─┤ X ├─░─
     ┌─┴─┐ ░        │   ░        │   ░ └─┬─┘ ░ 
q_5: ┤ X ├─░────────■───░────────┼───░───■───░─
     └───┘ ░            ░ ┌───┐┌─┴─┐ ░   │   ░ 
q_6: ──────░────────────░─┤ X ├┤ X ├─░───■───░─
           ░            ░ └───┘└───┘ ░       ░ 

           ░            ░            ░       ░ 
q_0: ──────░────────────░────────────░───■───░─
      